In [1]:
import numpy as np
import pandas as pd
import tensorflow
from matplotlib.pyplot import matplotlib
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.layers import Convolution2D,MaxPooling2D,ZeroPadding2D,Flatten,Dense,Activation,Dropout,Input
from keras.applications.imagenet_utils import preprocess_input
from keras import Sequential

from mtcnn import MTCNN


In [2]:
import os
import cv2
import pickle

names = os.listdir(r"C:\Users\hp\Desktop\data\dataset1")

filepath=r'C:\Users\hp\Desktop\data\dataset1'

filenames = []

for name in names:
    for file in os.listdir(os.path.join(filepath,name)):
        
        filenames.append(os.path.join(filepath,name,file))
        

pickle.dump(filenames,open('Filenames.pkl','wb'))

In [3]:
len(filenames)
filenames[300].split('\\')[-2]

'Mark Zuckerberg'

In [4]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

C:\Users\hp\anaconda3\anacond\Lib\site-packages\keras\src\layers\reshaping\zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
model.load_weights(r"C:\Users\hp\Downloads\vgg_face_weights.h5")


In [23]:
# pickle.dump(model,open('Model.pkl','wb'))

model.save('model.h5')

C:\Users\hp\anaconda3\anacond\Lib\site-packages\keras\src\models\model.py:342: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  warnings.warn(


In [9]:
def facial_feature_extractor(img_path,model):
#     img=face_extractor(img_path)
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img = np.expand_dims(img_array,axis=0)
    preprocessed_img = preprocess_input(expanded_img)

    result = model.predict(preprocessed_img).flatten()

    return result


In [13]:
def face_extractor(img_path):
    detector = MTCNN()
    # load img -> face detection
    img = cv2.imread(img_path)
    results = detector.detect_faces(img)

    x,y,width,height = results[0]['box']

    img = img[y:y+height+25,x:x+width+25]
    
    return img

    

In [14]:
from tqdm import tqdm

In [15]:
# features = []

# for file in tqdm(filenames):
#     features.append(facial_feature_extractor(file,model))

In [17]:
import pickle



# pickle.dump(features,open('features.pkl','wb'))

features = pickle.load(open(r"C:\Users\hp\Desktop\data\features.pkl",'rb'))



In [18]:
len(features[0])

2622

In [19]:

from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from PIL import Image



In [20]:
detector = MTCNN()
# load img -> face detectmyimg.jpgion
sample_img = cv2.imread(r"C:\Users\hp\Desktop\data\test_images\myimg.jpg")
results = detector.detect_faces(sample_img)

x,y,width,height = results[0]['box']

face = sample_img[y:y+height,x:x+width]

#  extract its features
image = Image.fromarray(face)
image = image.resize((224,224))

face_array = np.asarray(image)

face_array = face_array.astype('float32')

expanded_img = np.expand_dims(face_array,axis=0)
preprocessed_img = preprocess_input(expanded_img)
result = model.predict(preprocessed_img).flatten()
#print(result)
#print(result.shape)
# find the cosine distance of current image with all the 8655 features
similarity = []
for i in range(len(features)):
    similarity.append(cosine_similarity(result.reshape(1,-1),features[i].reshape(1,-1))[0][0])

index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]
print(max(similarity))
temp_img = cv2.imread(filenames[index_pos])
cv2.imshow('output',temp_img)
cv2.waitKey(0)
# recommend that image

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
0.6928188


-1

In [13]:
file_path = r"C:\Users\hp\Desktop\data\attendencesheet\name.csv.txt"
from datetime import datetime


In [35]:
cap=cv2.VideoCapture(0)




In [36]:
while True:
    
    ret,frame=cap.read()
    img=cv2.resize(frame,(224,224))
    
    
    
    detector=MTCNN()
    sample_img = frame
    results = detector.detect_faces(sample_img)
    
    x,y,w,h = results[0]['box']

    face = sample_img[y:y+h,x:x+w]
    
    #  extract its features
    image = Image.fromarray(face)
    image = image.resize((224,224))

    face_array = np.asarray(image)

    face_array = face_array.astype('float32')

    expanded_img = np.expand_dims(face_array,axis=0)
    preprocessed_img = preprocess_input(expanded_img)
    result = model.predict(preprocessed_img).flatten()
        
    similarity = []
    for i in range(len(features)):
        similarity.append(cosine_similarity(result.reshape(1,-1),features[i].reshape(1,-1))[0][0])

    index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]
    cosine = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][1]
    print(max(similarity))

    if cosine>0.40:
        temp_img = cv2.imread(filenames[index_pos])
        temp_img=cv2.resize(temp_img,(224,224))
        cv2.imshow('output',temp_img)


        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),thickness=2,color=(0,255,0))
        cv2.putText(frame,'Matched Succesfully [{}]'.format(filenames[index_pos].split('\\')[-2].upper()),(30,30),cv2.FONT_HERSHEY_TRIPLEX,1, (0,255,0), 1,cv2.LINE_AA)
        with open(file_path, mode='a', newline='\n') as file: 
            file.write('{}-{}\n'.format(filenames[index_pos].split('\\')[-2].upper(),datetime.now()))
                
    else:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),thickness=2,color=(0,0,255))
        cv2.putText(frame,'Not Matched ',(30,30),cv2.FONT_HERSHEY_TRIPLEX,1, (0,0,255), 1,cv2.LINE_AA)
        
        cv2.imshow('input',frame)
    
    
   
            
    cv2.imshow('input',frame)
    
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()       
cv2.destroyAllWindows()   

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
0.5514283
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 570ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

In [21]:

def predict(img):
    detector = MTCNN()
    # load img -> facload_img detection
    sample_img = cv2.imread(img)
    results = detector.detect_faces(sample_img)

    x,y,width,height = results[0]['box']

    face = sample_img[y:y+height,x:x+width]

    #  extract its features
    image = Image.fromarray(face)
    image = image.resize((224,224))

    face_array = np.asarray(image)

    face_array = face_array.astype('float32')

    expanded_img = np.expand_dims(face_array,axis=0)
    preprocessed_img = preprocess_input(expanded_img)
    result = model.predict(preprocessed_img).flatten()
    #print(result)
    #print(result.shape)
    # find the cosine distance of current image with all the 8655 features
    similarity = []
    for i in range(len(features)):
        similarity.append(cosine_similarity(result.reshape(1,-1),features[i].reshape(1,-1))[0][0])

    index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]
#     print(max(similarity))
    temp_img = cv2.imread(filenames[index_pos])
    temp_img=cv2.resize(temp_img,(224,224))
    
    cv2.putText(temp_img,'Matched !! [{}]'.format(filenames[index_pos].split('\\')[-2].upper()),(10,10),cv2.FONT_HERSHEY_TRIPLEX,0.35, (0,255,0), 1,cv2.LINE_AA)

    return temp_img

#     cv2.imshow('output',temp_img)
    cv2.waitKey(0)

In [27]:
def identify_people(img):
    file_path = r"C:\Users\hp\Desktop\data\attendencesheet\name.csv.txt"
    cap=cv2.VideoCapture(0)

    while True:
        
    
        ret,frame=cap.read()
        img=cv2.resize(frame,(224,224))



        detector=MTCNN()
        sample_img = frame
        results = detector.detect_faces(sample_img)

        x,y,w,h = results[0]['box']

        face = sample_img[y:y+h,x:x+w]

        #  extract its features
        image = Image.fromarray(face)
        image = image.resize((224,224))

        face_array = np.asarray(image)

        face_array = face_array.astype('float32')

        expanded_img = np.expand_dims(face_array,axis=0)
        preprocessed_img = preprocess_input(expanded_img)
        result = model.predict(preprocessed_img).flatten()


        similarity = []
        for i in range(len(features)):
            similarity.append(cosine_similarity(result.reshape(1,-1),features[i].reshape(1,-1))[0][0])

        index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]
        cosine = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][1]
        print(max(similarity))

        if cosine>0.5:
            temp_img = cv2.imread(filenames[index_pos])
            temp_img=cv2.resize(temp_img,(224,224))
#             cv2.imshow('output',temp_img)


            cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),thickness=2,color=(0,255,0))
            cv2.putText(frame,'Matched Succesfully [{}]'.format(filenames[index_pos].split('\\')[1].upper()),(30,30),cv2.FONT_HERSHEY_TRIPLEX,1, (0,255,0), 1,cv2.LINE_AA)
            frame=Image.fromarray(frame)
            return frame
#             return ('Matched Succesfully [{}]'.format(filenames[index_pos].split('\\')[1].upper())) 
#             with open(file_path, mode='a', newline='\n') as file: 
#                 file.write('{}-{}\n'.format(filenames[qindex_pos].split('\\')[1].upper(),datetime.now()))


        else:
            cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),thickness=2,color=(0,0,255))
            cv2.putText(frame,'Not Matched ',(30,30),cv2.FONT_HERSHEY_TRIPLEX,1, (0,0,255), 1,cv2.LINE_AA)
            frame=Image.fromarray(frame)
            return frame
            

#             cv2.imshow('input',frame)
            return 'Not Matched '




#         cv2.imshow('input',frame)


        if cv2.waitKey(1) & 0xFF==ord('q'):
            break

    cap.release()       
    cv2.destroyAllWindows()   

## Gradio App on Video


In [29]:
interface1=gr.Interface(
    fn=identify_people,
    inputs=gr.Image(sources=["webcam"]),
    outputs=gr.Image(type='numpy',image_mode='RGB'),
    title="Attendance Systems with Face Recognition",
    description='Open Camera By Clicking Start and Capture a Photo For Face Verification'
)

In [30]:
interface1.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.27.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://1a4d2ae7b4dd266894.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## gradio app on Image



In [24]:
import gradio as gr


In [31]:
interface=gr.Interface(
    fn=predict,
    inputs=gr.Image(type='filepath'),
    outputs=gr.Image(type='filepath'),
    title="Attendance Systems with Face Recognition",
    description='Open Camera By Clicking Start and Capture a Photo For Face Verification'
)

In [34]:
interface.launch(inline=False,share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://3a8d95ff9aac2e978b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [30]:
interface.close()

Closing server running on port: 7860
